<a href="https://colab.research.google.com/github/jwoonge/ML-algorithms/blob/master/10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   7281      0 --:--:-- --:--:-- --:--:--  7247
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


In [0]:
!pip install nltk
!pip install sklearn
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [0]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
import cupy as cp
from google.colab import drive

drive.mount('/content/gdrive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from zipfile import ZipFile
file_name = "movie_review.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
#'My_Drive/Colab_Notebooks/ML_assignments/10'
review_data = load_files(r"movie_review")
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
X_trian = cp.array(X_train) ; X_test = cp.array(X_test)
y_train = cp.array(y_train) ; y_test = cp.array(y_test)


1401


In [0]:
def sigmoid(input):
    return 1/(1+cp.exp(-input))
def sigmoid_d(input):
    return sigmoid(input)*(1-sigmoid(input))

In [0]:
def loss(h, label, weights, lam):
    pre = cp.sum(cp.average(-label * cp.log(h) - (1-label)*cp.log(1-h), axis=0))
    n = 0
    theta_sum = 0

    for weight in weights:
        n += weight.size
        theta_sum += cp.sum(cp.square(weight))
    reg = lam/(2*n)*theta_sum

    return pre + reg

In [0]:
def accuracy(h, label):
    predicted = cp.where(h[:,0]>0.5, 1,0)
    correct = predicted[predicted==label]
    return len(correct)/len(predicted)*100

In [0]:
class classifier:
    def __init__(self, shape, lam, learning_rate):
        self.weights = []
        for i in range(len(shape)-1):
            self.weights.append(cp.random.randn(shape[i]+1, shape[i+1]))
        self.learning_rate = learning_rate
        self.lam = lam
        self.num_layer = len(self.weights)
        self.n = 0
        for weight in self.weights:
            self.n += weight.size
    
    def forward_pass(self, input, weights=[]):
        if weights==[]:
            weights = self.weights
        values = []
        values.append(cp.array(np.insert(cp.asnumpy(input), 0, 1, axis=1)))
        values.append(cp.dot(values[-1], weights[0]))
        values.append(cp.array(np.insert(cp.asnumpy(sigmoid(values[-1])),0,1,axis=1)))
        for i in range(1, len(weights)-1):
            values.append(cp.dot(values[-1], weights[i]))
            values.append(cp.array(np.insert(cp.asnumpy(sigmoid(values[-1])),0,1,axis=1)))
        values.append(cp.dot(values[-1], weights[-1]))
        values.append(sigmoid(values[-1]))
        return values

    def gradient_descent(self, forward_values, label):
        update_values = []
        m = len(label)
        mult = (sigmoid_d(forward_values[-2]) * ((1-label)/(1-forward_values[-1]) - label/forward_values[-1])).T
        layer = len(self.weights)-1
        update_values.append(cp.dot(mult, forward_values[2*layer]).T)
        for i in range(1, len(self.weights)):
            layer = len(self.weights) - i - 1
            mult = (cp.dot(mult.T, self.weights[layer+1][1:,:].T) * (sigmoid_d(forward_values[layer*2+1]))).T
            update_values.append(cp.dot(mult, forward_values[2*layer]).T)
        for i in range(len(self.weights)):
            layer = len(self.weights)-i-1
            self.weights[layer] = self.weights[layer] - self.learning_rate * (update_values[i]/m + self.lam*self.weights[layer]/self.n)

    def optimize(self, data_train, label_train, data_test, label_test, epoch):
        forward_train = self.forward_pass(data_train)
        forward_test = self.forward_pass(data_test)
        h_train = forward_train[-1]
        h_test = forward_test[-1]
        loss_train = [loss(h_train, label_train, self.weights, self.lam)]
        loss_test = [loss(h_test, label_test, self.weights, self.lam)]
        accuracy_train = [accuracy(h_train, label_train)]
        accuracy_test = [accuracy(h_test, label_test)]
        self.best_train_accuracy = accuracy_train[-1]
        self.best_test_accuracy = accuracy_test[-1]
        self.best_train_loss = loss_train[-1]
        self.best_test_loss = loss_test[-1]
        self.best_weights = self.weights
        
        for i in range(epoch-1):

            self.gradient_descent(forward_train, label_train)
            forward_train = self.forward_pass(data_train)
            forward_test = self.forward_pass(data_test)
            h_train = forward_train[-1]
            h_test = forward_test[-1]
            loss_train.append(loss(h_train, label_train, self.weights, self.lam))
            loss_test.append(loss(h_test, label_test, self.weights, self.lam))
            accuracy_train.append(accuracy(h_train, label_train))
            accuracy_test.append(accuracy(h_test, label_test))
            
            if loss_test[-1] < self.best_test_loss:
                self.best_test_loss = loss_test[-1]
                self.best_train_loss = loss_train[-1]
                self.best_test_accuracy = accuracy_test[-1]
                self.best_train_accuracy = accuracy_train[-1]
                self.best_weights = self.weights

            print(i," LOS_T:", format(float(loss_train[-1]),'.5f'), "\tACC_T:", format(float(accuracy_train[-1]),'.5f'), "| LOS_V:", format(float(loss_test[-1]),'.5f'), "ACC_V:" ,format(float(accuracy_test[-1]),'.5f'), format(self.lam/self.n,'.10f'), self.n)
        return loss_train, accuracy_train, loss_test, accuracy_test
    
    def predict(self, datas):
        return self.forward_pass(datas, weights=self.best_weights)[-1]
    def predict_class(self, datas, best=True):
        if best:
            weights = self.best_weights
        else:
            weights = self.weights
        return cp.argmax(self.forward_pass(datas, weights=weights)[-1], axis=1)

In [0]:
Classifier = classifier([1500, 56, 24, 1], lam=27, learning_rate=1.5)
loss_train, accuracy_train, loss_test, accuracy_test = Classifier.optimize(X_train, cp.reshape(y_train,[len(y_train),1]), X_test, y_test, 200)

ValueError: ignored

In [0]:
print(np.shape(X_train))
print(len(y_train))
y_train = cp.reshape(y_train, (len(y_train),1))
print(np.shape(y_train))
#correct = h[np.abs(우리꺼 - 정답)<0.5]

(1401, 1500)
1401


TypeError: ignored